In [2]:
#调用torch库
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
#调用numpy库
import numpy as np
#调用matplotlib库
import matplotlib.pyplot as plt
#调用random库
import random

In [4]:
# 获取k折交叉验证某一折的训练集和验证集
def get_kfold_data(k, i, X, Y):
    fold_size = X.shape[0] // k
    val_start = i * fold_size
    if i != k - 1:
        val_end = (i + 1) * fold_size
        X_valid, Y_valid = X[val_start:val_end], Y[val_start:val_end]
        X_train = torch.cat([X[0:val_start], X[val_end:]], dim=0)
        Y_train = torch.cat([Y[0:val_start], Y[val_end:]], dim=0)
    else:
        X_valid, Y_valid = X[val_start:], Y[val_start:]
        X_train = X[0:val_start]
        Y_train = Y[0:val_start]

    return X_train, Y_train, X_valid, Y_valid

def k_fold_1(k, X_train, Y_train, train_function, para_list):
    train_loss_sum, valid_loss_sum = 0, 0
    train_acc_sum, valid_acc_sum = 0, 0

    for i in range(k):
        print('第', i + 1, '折验证结果: ')
        data = get_kfold_data(k, i, X_train, Y_train)
        train_loss, val_loss = train_function(data, para_list)
        print('训练损失：{:.4f}，验证损失：{:.4f}'.format(train_loss, val_loss))

        train_loss_sum += train_loss
        valid_loss_sum += val_loss

    print('最终k折交叉验证结果:')
    print('平均训练损失：{:.4f}，平均验证损失：{:.4f}'.format(train_loss_sum/k, valid_loss_sum/k))

回归模型十折交叉验证

In [5]:
# 数据生成
n_data = torch.zeros(10000, 500)
X = torch.normal(mean=n_data, std=n_data + 0.01)

n_noise = torch.zeros(10000, 1)
noise = torch.normal(mean=n_noise, std=n_noise + 0.01)

Y = 0.028 + 0.056*torch.sum(X, dim=1).unsqueeze(1) + noise

In [6]:
#使用课上给出的数据读取函数
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
        yield features.index_select(0, j), labels.index_select(0, j)

In [7]:
# 模型
class Feedforward(nn.Module):
  def __init__(self, feature, hidden):
    super(Feedforward, self).__init__()
    self.hidden = nn.Linear(feature, hidden)
    self.out = nn.Linear(hidden, 1)
 
  def forward(self, x):
    x = self.hidden(x)
    y_hat = self.out(x)
    return y_hat

In [8]:
# 超参数设置
device = 'cuda:0'
epoch_num = 5
lr = 0.01
feature_dim = 500
hidden_dim = 100
batch_size = 128
# 模型、优化器以及损失函数设置
loss = torch.nn.MSELoss(reduction='mean')

# 配置列表
para_list = [device, epoch_num, lr, feature_dim, hidden_dim, batch_size, loss]

In [12]:
# 适配后的训练函数
def train_function(data, para_list):
    x_train, y_train, x_valid, y_valid = data
    device, epoch_num, lr, feature_dim, hidden_dim, batch_size, loss = para_list
    model = Feedforward(feature_dim, hidden_dim).to(device)
    opt = torch.optim.SGD(model.parameters(), lr = lr)

    train_loss_list = []
    test_loss_list = []

    for epoch in range(epoch_num):
        for x, y in data_iter(batch_size, x_train, y_train):
            x, y = x.to(device), y.to(device)
            y_hat = model.forward(x)
            l = loss(y_hat, y)
            opt.zero_grad()
            l.backward()
            opt.step()

            train_loss_list.append(l.item())
    
    with torch.no_grad():
        total_loss = 0
        counter = 0
        for x, y in data_iter(batch_size, x_valid, y_valid):
            test_x, test_y = x.to(device), y.to(device)
            test_pred = model.forward(test_x)

            l = loss(test_pred, test_y).sum()
            total_loss += l
            counter += 1
        test_loss_list.append(total_loss.item())     

    train_loss = np.array(train_loss_list).mean()
    val_loss = np.array(test_loss_list).mean()
    return train_loss, val_loss

In [13]:
k_fold_1(10, X, Y, train_function, para_list)

第 1 折验证结果: 
训练损失：0.0004，验证损失：0.0020
第 2 折验证结果: 
训练损失：0.0013，验证损失：0.0021
第 3 折验证结果: 
训练损失：0.0013，验证损失：0.0021
第 4 折验证结果: 
训练损失：0.0003，验证损失：0.0019
第 5 折验证结果: 
训练损失：0.0010，验证损失：0.0019
第 6 折验证结果: 
训练损失：0.0005，验证损失：0.0023
第 7 折验证结果: 
训练损失：0.0004，验证损失：0.0023
第 8 折验证结果: 
训练损失：0.0003，验证损失：0.0021
第 9 折验证结果: 
训练损失：0.0003，验证损失：0.0022
第 10 折验证结果: 
训练损失：0.0003，验证损失：0.0021
最终k折交叉验证结果:
平均训练损失：0.0006，平均验证损失：0.0021


二分类模型十折交叉验证

In [20]:
# 数据生成
n_data = torch.zeros(10000, 200)
X1 = torch.normal(mean=n_data + 1, std=n_data + 0.01)
Y1 = torch.zeros(10000, 1)

X2 = torch.normal(mean=n_data - 1, std=n_data + 0.01)
Y2 = torch.ones(10000, 1)

X = torch.cat([X1, X2], dim=0)
Y = torch.cat([Y1, Y2], dim=0)

In [21]:
#使用课上给出的数据读取函数
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices) # 样本的读取顺序是随机的
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # 最后一次可能不足一个batch
        yield features.index_select(0, j), labels.index_select(0, j)

In [22]:
# 模型定义
class Feedforward(nn.Module):
  def __init__(self, feature, hidden):
    super(Feedforward, self).__init__()
    self.hidden = nn.Linear(feature, hidden)
    self.out = nn.Linear(hidden, 1)
    self.act = nn.Sigmoid()
 
  def forward(self, x):
    x = self.hidden(x)
    x = self.out(x)
    return self.act(x)

In [23]:
# 超参数设置
device = 'cuda:0'
epoch_num = 100
lr = 0.01
feature_dim = 200
hidden_dim = 100
batch_size = 128
# 模型、优化器以及损失函数设置
loss = torch.nn.BCELoss()

# 配置列表
para_list = [device, epoch_num, lr, feature_dim, hidden_dim, batch_size, loss]

In [24]:
# 适配后的训练函数
def train_function(data, para_list):
    x_train, y_train, x_valid, y_valid = data
    device, epoch_num, lr, feature_dim, hidden_dim, batch_size, loss = para_list
    model = Feedforward(feature_dim, hidden_dim).to(device)
    opt = torch.optim.SGD(model.parameters(), lr = lr)

    train_loss_list = []
    test_loss_list = []

    for epoch in range(epoch_num):
        for x, y in data_iter(batch_size, x_train, y_train):
            x, y = x.to(device), y.to(device)
            y_hat = model.forward(x)
            l = loss(y_hat, y)
            opt.zero_grad()
            l.backward()
            opt.step()

            train_loss_list.append(l.item())
    
    with torch.no_grad():
        total_loss = 0
        counter = 0
        for x, y in data_iter(batch_size, x_valid, y_valid):
            test_x, test_y = x.to(device), y.to(device)
            test_pred = model.forward(test_x)

            l = loss(test_pred, test_y).sum()
            total_loss += l
            counter += 1
        test_loss_list.append(total_loss.item())     

    train_loss = np.array(train_loss_list).mean()
    val_loss = np.array(test_loss_list).mean()
    return train_loss, val_loss

In [25]:
k_fold_1(10, X, Y, train_function, para_list)

第 1 折验证结果: 
训练损失：0.0006，验证损失：0.0004
第 2 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 3 折验证结果: 
训练损失：0.0004，验证损失：0.0004
第 4 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 5 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 6 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 7 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 8 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 9 折验证结果: 
训练损失：0.0005，验证损失：0.0004
第 10 折验证结果: 
训练损失：0.0005，验证损失：0.0004
最终k折交叉验证结果:
平均训练损失：0.0005，平均验证损失：0.0004


多分类模型十折交叉验证

In [3]:
# 操作一下这个数据集
train_dataset = torchvision.datasets.MNIST(root="../Datasets/MNIST", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root="../Datasets/MNIST", train=False, transform=transforms.ToTensor())

num_workers = 16

In [4]:
# 获取k折交叉验证某一折的训练集和验证集
def get_kfold_data(k, i, train_dataset, test_dataset):
    dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])
    dataset_list = [dataset for dataset in torch.utils.data.random_split(dataset, [len(dataset) // k for i in range(k)])]
    if i != k-1:
        train_list = dataset_list[0:i] + dataset_list[i+1:]
    else:
        train_list = dataset_list[0:i]

    train_dataset = torch.utils.data.ConcatDataset(train_list)
    valid_dataset = dataset_list[i]

    return [train_dataset, valid_dataset]

def k_fold_2(k, train_dataset, test_dataset, train_function, para_list):
    train_loss_sum, valid_loss_sum = 0, 0
    train_acc_sum, valid_acc_sum = 0, 0

    for i in range(k):
        print('第', i + 1, '折验证结果: ')
        datasets = get_kfold_data(k, i, train_dataset, test_dataset)
        train_loss, val_loss, val_acc = train_function(datasets, para_list)
        print('训练损失：{:.4f}，验证损失：{:.4f}，验证精度：{:.3f}%'.format(train_loss, val_loss, val_acc))

        train_loss_sum += train_loss
        valid_loss_sum += val_loss
        valid_acc_sum += val_acc

    print('最终k折交叉验证结果:')
    print('平均训练损失：{:.4f}，平均验证损失：{:.4f}，平均验证精度：{:.3f}%'.format(train_loss_sum/k, valid_loss_sum/k, valid_acc_sum/k))

In [5]:
# 模型定义
class FlattenLayer(torch.nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x):
        return x.view(x.shape[0], -1)

class Feedforward(nn.Module):
    def __init__(self, image_size, class_num, layer_num, hidden_dim, activation, drop_out=0):
        super(Feedforward, self).__init__()

        self.flatten = FlattenLayer()
        self.linear_in = nn.Linear(image_size*image_size, hidden_dim)
        hidden_layers = []
        hidden_layers.append(activation)
        if drop_out > 0:
            hidden_layers.append(nn.Dropout(p=drop_out))
        
        for _ in range(layer_num - 1):
            hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
            hidden_layers.append(activation)
            if drop_out > 0:
                hidden_layers.append(nn.Dropout(p=drop_out))
        self.hidden_layers = nn.Sequential(*hidden_layers)
        self.linear_out = nn.Linear(hidden_dim, class_num)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_in(x)
        x = self.hidden_layers(x)
        x = self.linear_out(x)
        return self.softmax(x)

In [10]:
# 超参数设置
epoch_num = 20
lr = 0.03
image_size = 28
class_num = 10
device = 'cuda:0'
batch_size = 32

layer_num  = 1   # 隐藏层数为1层
hidden_dim = 100 # 隐藏层单元数为100
drop_out   = 0   # 不设置dropout
lambd      = 0   # 不设置正则化项
activation = nn.ReLU() # 激活函数设置为ReLU
loss = nn.CrossEntropyLoss()

para_list = [epoch_num, lr, image_size, class_num, device, layer_num, hidden_dim, drop_out, lambd, activation, loss]

In [11]:
# 计算预测的准确度
def accuracy(y_hat, y):  #@save
    y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [14]:
# 适配后的训练函数
def train_function(datasets, para_list):
    train_dataset, valid_dataset = datasets
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    epoch_num, lr, image_size, class_num, device, layer_num, hidden_dim, drop_out, lambd, activation, loss = para_list
    model = Feedforward(image_size=image_size, class_num=class_num, layer_num=layer_num, hidden_dim=hidden_dim, activation=activation, drop_out=drop_out).to(device)
    opt = torch.optim.SGD(model.parameters(), lr = lr, weight_decay=lambd)

    train_loss_list = []
    test_loss_list = []
    test_acc_list = []

    for epoch in range(epoch_num):
        for i, datas in enumerate(train_loader):
            train_x, train_label = datas[0].to(device), datas[1].to(device)
            train_pred = model.forward(train_x)
            l = loss(train_pred, train_label).sum()
            opt.zero_grad()
            l.backward()
            opt.step()
            train_loss_list.append(l.item())
    
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        for i, datas in enumerate(valid_loader):
            test_x, test_label = datas[0].to(device), datas[1].to(device)
            test_pred = model.forward(test_x)

            l = loss(test_pred, test_label).sum()
            total_loss += l

            acc = accuracy(test_pred, test_label) / len(test_pred)
            total_acc += acc

        test_loss_list.append(total_loss.item())
        test_acc_list.append(acc)

    train_loss = np.array(train_loss_list).mean()
    val_loss = np.array(test_loss_list).mean()
    val_acc = np.array(test_acc_list).mean()
    return train_loss, val_loss/100, val_acc*100

In [15]:
k_fold_2(10, train_dataset, test_dataset, train_function, para_list)

第 1 折验证结果: 


/tmp/ipykernel_960443/1595188733.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(x)


训练损失：1.5724，验证损失：3.3506，验证精度：91.667%
第 2 折验证结果: 
训练损失：1.5744，验证损失：3.3351，验证精度：91.667%
第 3 折验证结果: 
训练损失：1.5716，验证损失：3.3439，验证精度：87.500%
第 4 折验证结果: 
训练损失：1.5751，验证损失：3.3605，验证精度：95.833%
第 5 折验证结果: 
训练损失：1.5877，验证损失：3.3567，验证精度：95.833%
第 6 折验证结果: 
训练损失：1.6150，验证损失：3.3627，验证精度：95.833%
第 7 折验证结果: 
训练损失：1.5740，验证损失：3.3344，验证精度：95.833%
第 8 折验证结果: 
训练损失：1.5688，验证损失：3.3491，验证精度：91.667%
第 9 折验证结果: 
训练损失：1.5704，验证损失：3.3518，验证精度：91.667%
第 10 折验证结果: 
训练损失：1.5806，验证损失：3.3531，验证精度：95.833%
最终k折交叉验证结果:
平均训练损失：1.5790，平均验证损失：3.3498，平均验证精度：93.333%
